Import Data:

In [28]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_columns', 999)
import pandas.io.sql as psql
# plot a figure directly on Notebook
import matplotlib.pyplot as plt

In [29]:
admissions = pd.read_csv("data/ADMISSIONS.csv.gz", compression='gzip')
diagnoses = pd.read_csv("data/DIAGNOSES_ICD.csv.gz", compression='gzip')
patients = pd.read_csv("data/PATIENTS.csv.gz", compression='gzip')

# admissions = pd.read_csv("demo-data/ADMISSIONS.csv")
# diagnoses = pd.read_csv("demo-data/DIAGNOSES_ICD.csv")
# patients = pd.read_csv("demo-data/PATIENTS.csv")

print("Admissions Table\t" + str(admissions.shape))
print(admissions.columns)
print("\nDiagnoses Table\t\t" + str(diagnoses.shape))
print(diagnoses.columns)
print("\nPatients Table\t\t" + str(patients.shape))
print(patients.columns)

Admissions Table	(58976, 19)
Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')

Diagnoses Table		(651047, 5)
Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

Patients Table		(46520, 8)
Index(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG'],
      dtype='object')


In [30]:
# # Iterate through table
# # Regular Table: subect_id: the whole row that is their first admssion
# # During iteration, if subject_id is already in the newTableWeAreCreating, then we have seen the subject, we are doing a readmission right now
#     # But, we have to check the time frame, because if its 30 days after the dictionary value, this is a new set of admission to consider
# # first_admission_dataframe = pd.DataFrame(columns=admission_table.columns)
# first_admission_dataframe = pd.DataFrame(columns=admission_table.columns)

# # Iterate over rows
# for index, row in admission_table.iterrows():
#     # check if the subject_id has been admitted before
#     if row['subject_id'] not in first_admission_dataframe['subject_id'].values:
#         # Append the row to the new DataFrame
#         first_admission_dataframe = pd.concat([first_admission_dataframe, pd.DataFrame([row])], 
#                                               axis=0, ignore_index=True)

# print(first_admission_dataframe['admittime'])
# # # Reset the index of the new DataFrame
# # first_admission_dataframe.reset_index(drop=True, inplace=True)

In [31]:
# Merge Tables
# merged = admissions.merge(patients, on="SUBJECT_ID").merge(diagnoses, 
#             left_on=["SUBJECT_ID","HADM_ID"], right_on=["SUBJECT_ID","HADM_ID"])
merged = admissions

#
# PROBLEM: there are multiple entries now for one admission time, each entry represents a different diagnoses
    # Counts for diagnoses for one admittime range from 1 to 57
        # print(merged['ADMITTIME'].value_counts())
# Possible Fixes: merge them all together and make a new column that holds each diagnoses in an array

# Convert time to objects
merged["ADMITTIME"] = pd.to_datetime(merged["ADMITTIME"], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
merged["DISCHTIME"] = pd.to_datetime(merged["DISCHTIME"], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
merged["DEATHTIME"] = pd.to_datetime(merged["DISCHTIME"], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

merged = merged.sort_values(['SUBJECT_ID','ADMITTIME'])
merged = merged.reset_index(drop = True)

merged['NEXT_ADMITTIME'] = merged.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
# get the next admission type
merged['NEXT_ADMISSION_TYPE'] = merged.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

In [32]:
merged['NEXT_ADMITTIME'].unique()

<DatetimeArray>
[                'NaT', '2135-05-09 14:11:00', '2135-01-30 20:50:00',
 '2157-10-18 19:34:00', '2191-02-23 05:23:00', '2131-05-12 19:49:00',
 '2134-05-10 11:30:00', '2119-01-04 18:12:00', '2157-12-02 00:45:00',
 '2174-01-04 22:21:00',
 ...
 '2191-12-23 01:03:00', '2133-08-01 19:20:00', '2125-11-01 00:00:00',
 '2197-06-16 03:01:00', '2197-07-03 02:24:00', '2132-09-15 00:36:00',
 '2182-07-03 19:50:00', '2201-05-15 13:12:00', '2157-01-05 17:27:00',
 '2157-02-16 17:31:00']
Length: 12451, dtype: datetime64[ns]

In [33]:
rows = merged.NEXT_ADMISSION_TYPE == 'ELECTIVE'
merged.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
merged.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN
merged['NEXT_ADMITTIME'].unique()

<DatetimeArray>
[                'NaT', '2135-05-09 14:11:00', '2135-01-30 20:50:00',
 '2157-10-18 19:34:00', '2191-02-23 05:23:00', '2131-05-12 19:49:00',
 '2119-01-04 18:12:00', '2157-12-02 00:45:00', '2174-01-04 22:21:00',
 '2196-04-14 04:02:00',
 ...
 '2191-08-03 19:11:00', '2191-12-23 01:03:00', '2133-08-01 19:20:00',
 '2125-11-01 00:00:00', '2197-06-16 03:01:00', '2197-07-03 02:24:00',
 '2182-07-03 19:50:00', '2201-05-15 13:12:00', '2157-01-05 17:27:00',
 '2157-02-16 17:31:00']
Length: 11048, dtype: datetime64[ns]

In [34]:
merged = merged.sort_values(['SUBJECT_ID','ADMITTIME'])
merged[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = merged.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')

In [35]:
merged['NEXT_ADMITTIME'].unique()

<DatetimeArray>
[                'NaT', '2135-05-09 14:11:00', '2135-01-30 20:50:00',
 '2157-10-18 19:34:00', '2191-02-23 05:23:00', '2131-05-12 19:49:00',
 '2119-01-04 18:12:00', '2157-12-02 00:45:00', '2174-01-04 22:21:00',
 '2196-04-14 04:02:00',
 ...
 '2191-08-03 19:11:00', '2191-12-23 01:03:00', '2133-08-01 19:20:00',
 '2125-11-01 00:00:00', '2197-06-16 03:01:00', '2197-07-03 02:24:00',
 '2182-07-03 19:50:00', '2201-05-15 13:12:00', '2157-01-05 17:27:00',
 '2157-02-16 17:31:00']
Length: 11048, dtype: datetime64[ns]

In [37]:
merged['DAYS_NEXT_ADMIT']=(merged.NEXT_ADMITTIME - merged.DISCHTIME).dt.total_seconds()/(24*60*60)
merged['DAYS_STAY'] = (merged.DISCHTIME - merged.ADMITTIME).dt.total_seconds()/(24*60*60)
merged

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,NEXT_ADMITTIME,NEXT_ADMISSION_TYPE,DAYS_NEXT_ADMIT,DAYS_STAY
0,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,2138-07-21 15:48:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,NaT,NaN,NaN,3.863889
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-31 13:58:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1,NaT,NaN,NaN,10.784722
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,2191-03-23 18:41:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1,NaT,NaN,NaN,7.759028
3,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,2103-02-04 12:15:00,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaN,NaN,NEWBORN,0,1,NaT,NaN,NaN,2.322222
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,2175-06-15 16:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1,NaT,NaN,NaN,16.364583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58972,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,2181-02-12 17:05:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,JEWISH,MARRIED,WHITE,2181-01-26 23:35:00,2181-01-27 04:18:00,FEVER,0,1,NaT,NaN,NaN,16.595833
58972,58973,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,2185-01-05 12:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,DIVERTICULITIS/SDA,0,1,NaT,NaN,NaN,12.156250
58973,58974,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,2144-07-28 17:56:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2144-07-25 13:40:00,2144-07-25 18:50:00,RETROPERITONEAL HEMORRHAGE,0,1,NaT,NaN,NaN,2.995139
58974,58975,99995,137810,2147-02-08 08:00:00,2147-02-11 13:15:00,2147-02-11 13:15:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,ABDOMINAL AORTIC ANEURYSM/SDA,0,1,NaT,NaN,NaN,3.218750


Case to consider: If there are three admission back to back less than 30 days apart

Important features:

    Admissions table
        diagnosis
        admission_type